In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import scipy.stats as st
from pathlib import Path


In [5]:
#pull in CSV df - 2019 through 2021 - vehicle csv

In [ ]:
#clean data for CA (Legal) -  years 2019 through 2021 - MONTHNAME - ACC_TYPE - OWNER - MAK_MODNAME- BODY_TYPNAME - HARM_EVNAME - MAN_COLLNAME - VNUM_LAN(exculed 1 lane)

In [ ]:
#clean data for (Somewhat Legal) - years 2019 through 2021 - MONTHNAME - ACC_TYPE - OWNER - MAK_MODNAME- BODY_TYPNAME - HARM_EVNAME - MAN_COLLNAME - VNUM_LAN(exculed 1 lane)

In [ ]:
#clean data for (Illegal) - years 2019 through 2021 - MONTHNAME - ACC_TYPE - OWNER - MAK_MODNAME- BODY_TYPNAME - HARM_EVNAME - MAN_COLLNAME - VNUM_LAN(exculed 1 lane)

In [ ]:
#clean data for (Not Mentioned) - years 2019 through 2021 - MONTHNAME - ACC_TYPE - OWNER - MAK_MODNAME- BODY_TYPNAME - HARM_EVNAME - MAN_COLLNAME - VNUM_LAN(exculed 1 lane)

In [ ]:
# Motorcycle type affects safety of lane splitting (Calvin)

In [ ]:
# Motorcycle laws affects fewer crashes (Rail)

In [ ]:
# Motorcycle laws affects deadly crashes (Jack)

In [ ]:
# Motorcycle  laws affects injuries (Rachael)

In [ ]:
#compare UT years accidents with chart 

In [ ]:
#compare ID years accidents with chart 

In [ ]:
#compare UT & ID to CA with chart

In [ ]:
#compare COLL types 